In [ ]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tqdm import tqdm
from PIL import Image

In [ ]:
df=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")

In [ ]:
df.head()

In [ ]:
list_images=os.listdir('../input/aptos2019-blindness-detection/train_images')

In [ ]:
list_images[:5]

In [ ]:
len(list_images)

In [ ]:
df.shape

In [ ]:
df['diagnosis'].unique()

In [ ]:
import matplotlib.image as implt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#import random
#k=1
#plt.subplots(5,2,figsize=(20,20))
#for i in range(10):
#    path='../input/eye-diabetic/eye-cotton/'+list_images[i]
#    img=implt.imread(path)
#    plt.subplot(5,2,i+1)
#    plt.imshow(img)

In [ ]:
import random
k=1
plt.subplots(5,2,figsize=(20,20))
for i in range(5):
    temp=df[df['diagnosis']==i].copy()
    for j in range(2):
        image=random.choice(list(temp['id_code']))
        path='../input/aptos2019-blindness-detection'+'/train_images/'+image+'.png'
        img=implt.imread(path)
        plt.subplot(5,2,k)
        k+=1
        plt.imshow(img)
        plt.title(f"the class is {i}")
        plt.axis(False)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
import cv2 

In [ ]:
import seaborn as sn

In [ ]:
img=cv2.imread('../input/aptos2019-blindness-detection/train_images/356304d15a5c.png')
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
sn.distplot(img)


In [ ]:
sn.distplot(img)
plt.xlim(-1,25)

In [ ]:
def preprocess(imgpath):
    img=cv2.imread(imgpath)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_black=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    mask=img_black>5
    temp=img[np.ix_(mask.any(1),mask.any(0))]
    if temp.shape[0]==0:
        clean_img=img
    else:
        img0=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
        img1=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
        img2=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        clean_img=np.stack([img0,img1,img2],axis=-1)
    image_final=cv2.addWeighted(clean_img,0.8,cv2.GaussianBlur(clean_img,(15,15),30),0.2,80)
    yield image_final
        

In [ ]:
path='../input/aptos2019-blindness-detection/train_images/'+list_images[1]
name=list_images[1]
image=next(preprocess(path))
#image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)


In [ ]:
plt.imshow(image)

In [ ]:
#os.mkdir('eye-cotton')

In [ ]:
#for i in tqdm(range(len(list_images))):
#    path='../input/aptos2019-blindness-detection/train_images/'+list_images[i]
#    name=list_images[i]
#    image=next(preprocess(path))
#    newpath='eye-cotton/'+list_images[i]
#    Image.fromarray(image).save(newpath)

In [ ]:
##sizes are uneven
#def get_size(x):
#    x= '../input/aptos2019-blindness-detection/train_images/'+x+'.png'
#    img=load_img(x)
#    img=img_to_array(img)
#    return img.shape
#df['sizes']=df['id_code'].apply(get_size)

In [ ]:
df.head()

In [ ]:
#print(f"the largesize is {np.max(df['sizes'])} and small size is {np.min(df['sizes'])}")

In [ ]:
import seaborn as sn

In [ ]:
#widths=df['sizes'].apply(lambda x: x[0])
#sn.distplot(widths)

In [ ]:
#widths[widths>1500].shape

In [ ]:
#del widths

In [ ]:
#heights=df['sizes'].apply(lambda x:x[1])
#sn.distplot(heights)

In [ ]:
#del heights

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
from tqdm import tqdm

In [ ]:
df['diagnosis'].hist()

In [ ]:
df.sample(frac=1).reset_index(inplace=True,drop=True)
df.head()

In [ ]:
df['id_code']=df['id_code'].apply(lambda x: x+'.png')

In [ ]:
x=df['id_code'].values
y=df['diagnosis'].values

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5)
df['kfold']=-1
for i,(train,test) in enumerate(skf.split(x,y)):
    df.loc[test,'kfold']=i

In [ ]:
df['kfold'].value_counts()

In [ ]:
cols=[f'label{i}' for i in range(5)]

In [ ]:
cols

In [ ]:
data=pd.DataFrame(to_categorical(df['diagnosis'].values,num_classes=5),columns=cols)

In [ ]:
df=df.join(data)

In [ ]:
df.drop('diagnosis',axis=1,inplace=True)

In [ ]:
train_data=df[df['kfold']!=0].drop('kfold',axis=1).reset_index(drop=True)
test_data=df[df['kfold']==0].drop('kfold',axis=1).reset_index(drop=True)

In [ ]:
train_data.head()

In [ ]:
neg_weights=[]
pos_weights=[]
for i in range(5):
    neg_weights.append(np.sum(train_data.iloc[:,i+1].values,axis=0)/train_data.shape[0])
    pos_weights.append(np.sum(1-train_data.iloc[:,i+1].values,axis=0)/train_data.shape[0])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Input,BatchNormalization,Dropout,ReLU,MaxPooling2D
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import EfficientNetB7,Xception

In [ ]:
pre_model=DenseNet201(input_shape=(900,900,3),include_top=False,weights='imagenet')
pre_model.summary()

In [ ]:
pre_model.layers[-9].name.split('_')[-1]

In [ ]:
len(pre_model.layers)

In [ ]:
for i in range(len(pre_model.layers)):
    if i>=707-9:
        if pre_model.layers[i].name.split('_')[-1]=='bn':
            pre_model.layers[i].trainable=False
        else:
            pre_model.layers[i].trainable=True
    else:
        pre_model.layers[i].trainable=False
#    if i>=707-9:
#        pre_model.layers[i].trainable=True
#    else:
#        pre_model.layers[i].trainable=False

#        

In [ ]:
#prelayer=premodel.layers[-1].output
#x=GlobalAveragePooling2D()(prelayer)
#x=Dense(800,activation='relu')(x)
#outs=Dense(5,activation='softmax')(x)

In [ ]:
#pre_model=Xception(include_top=False,weights='imagenet',
#                          input_shape=(900,900,3))
#pre_model.summary()


In [ ]:
import math

In [ ]:
x=pre_model.layers[-1].output

In [ ]:
class SPP(tf.keras.layers.Layer):
    def __init__(self,pool_list):
        super(SPP,self).__init__()
        self.pool_list=pool_list
        self.flatten=tf.keras.layers.Flatten()
        self.concat=tf.keras.layers.Concatenate()
    def get_config(self,):
        config = super().get_config().copy()
        config.update({'self.pool_list':[1,2,4],
                      'self.flatten':tf.keras.layers.Flatten(),
                      'self.concat':tf.keras.layers.Concatenate()})
        return config
        
    def call(self,inputs):
        shape=inputs.get_shape().as_list()
        self.output_list=[]
        for i in self.pool_list:
            pool=int(math.ceil(shape[1]/i))
            stride=int(math.floor(shape[1]/i))
            x=MaxPooling2D((pool,pool),strides=(stride,stride))(inputs)
            x=self.flatten(x)
            self.output_list.append(x)
        output=self.concat(self.output_list)
        return output
        
        
            

In [ ]:
spp=SPP([1,2,4])

In [ ]:
#pre_model.layers[126].name.split('_')[0][-2:]

In [ ]:
#for i in range(len(pre_model.layers)):
#    if pre_model.layers[i].name.split('_')[0][-2:]=='14':
#        if pre_model.layers[i].name.split('_')[-1]=='bn':
#            pre_model.layers[i].trainable=False
#        else:
#            pre_model.layers[i].trainable=True
#            
#    #if i>=126:
#        #pre_model.layers[i].trainable=True
#    else:
#        pre_model.layers[i].trainable=False

In [ ]:
import tensorflow as tf

In [ ]:
x=pre_model.layers[-1].output
x=spp(x)
x=Dropout(0.3)(x)
x=Dense(1024,kernel_initializer='he_normal')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Dense(512,kernel_initializer='he_normal')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Dense(128,kernel_initializer='he_normal')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
outs=Dense(5,activation='softmax')(x)

In [ ]:
#x=pre_model.layers[-1].output
#x=GlobalMaxPooling2D()(x)
#x=Dropout(0.3)(x)
#x=Dense(1024,kernel_initializer='he_normal')(x)
#x=BatchNormalization()(x)
#x=ReLU()(x)
#x=Dense(512,kernel_initializer='he_normal')(x)
#x=BatchNormalization()(x)
#x=ReLU()(x)
#x=Dense(128,kernel_initializer='he_normal')(x)
#x=BatchNormalization()(x)
#x=ReLU()(x)
#outs=Dense(5,activation='softmax')(x)

In [ ]:
model=Model(pre_model.input,outs)

In [ ]:
model.summary()

In [ ]:
pos_weights=np.array(pos_weights)
neg_weights=np.array(neg_weights)

In [ ]:
pos_weights

In [ ]:
neg_weights

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
traingen=ImageDataGenerator(rescale=1./255,horizontal_flip=True,brightness_range=[0.7,1.5],rotation_range=60)
validgen=ImageDataGenerator(rescale=1./255)

In [ ]:
#train_data['id_code']=train_data['id_code'].apply(lambda x: '../input/preprocessedimages/eye-cotton/'+x)
#test_data['id_code']=test_data['id_code'].apply(lambda x: '../input/preprocessedimages/eye-cotton/'+x)

In [ ]:
train=traingen.flow_from_dataframe(
                                    dataframe=train_data,
                                    batch_size=29,
                                    directory='../input/preprocessedimages/eye-cotton/'
                                    ,x_col='id_code',
                                    y_col=cols,
                                    target_size=(900,900),
                                    color_mode='rgb',
                                    class_mode='raw')

In [ ]:
valid=validgen.flow_from_dataframe(
                                    dataframe=test_data,
                                    directory='../input/preprocessedimages/eye-cotton/'
                                    ,x_col='id_code',
                                    y_col=cols,
                                    target_size=(900,900),
                                    batch_size=1,
                                    color_mode='rgb',
                                    class_mode='raw')

In [ ]:
import tensorflow.keras.backend as K

In [ ]:
images=next(iter(train))

In [ ]:
images=images[0]

In [ ]:
plt.subplots(10,3,figsize=(20,20))
for i in range(len(images)):
    plt.subplot(10,3,i+1)
    image=images[i]
    plt.imshow(image)
    plt.axis(False)

In [ ]:
def custom_loss(y_true,y_pred):
    y_true=tf.cast(y_true,dtype=y_pred.dtype)
    loss=0.0
    pos_loss=0.0
    neg_loss=0.0
    for i in range(5):
        pos_loss+=-1*K.mean(pos_weights[i]*y_true[i,:]*K.log(y_pred[i,:]+1e-5))
        neg_loss+=-1*K.mean(neg_weights[i]*(1-y_true[i,:])*K.log(1-y_pred[i,:]+1e-5))
    loss=pos_loss+neg_loss
    return loss

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=tf.keras.metrics.AUC(curve='PR'))

In [ ]:
def lr_fn(epoch):
    if epoch<5:
        return 0.0008*(1.25)**(epoch/5)
    else:
        return 0.001*(0.5)**((epoch-5)/10)
lr_schedule=tf.keras.callbacks.LearningRateScheduler(lambda epoch:lr_fn(epoch),verbose=1)

In [ ]:
plt.plot(list(range(30)),list(lr_fn(i) for i in range(30)))

In [ ]:
saving=tf.keras.callbacks.ModelCheckpoint(
                                           filepath='dense_spp.h5',
                                            monitor="val_loss",
                                           verbose=1,
                                           save_best_only=True,
                                           save_weights_only=False,
                                           mode="min",
                                           save_freq="epoch",)
early=tf.keras.callbacks.EarlyStopping(
                                       monitor="val_loss",
                                       patience=5,
                                       verbose=1,
                                       mode="min",)
#lr=tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001*(0.5)**(epoch/10),verbose=1)

In [ ]:
model.fit(train,validation_data=valid,epochs=40,callbacks=[saving,lr_schedule])

In [ ]:
class SPP(tf.keras.layers.Layer):
    def __init__(self,pool_list=[1,2,4],**kwargs):
        super(SPP,self).__init__()
        self.pool_list=pool_list
        self.flatten=tf.keras.layers.Flatten()
        self.concat=tf.keras.layers.Concatenate()
    def get_config(self,):
        config = super().get_config().copy()
        config.update({'self.pool_list':[1,2,4],
                      'self.flatten':tf.keras.layers.Flatten(),
                      'self.concat':tf.keras.layers.Concatenate()})
        return config
        
    def call(self,inputs):
        shape=inputs.get_shape().as_list()
        self.output_list=[]
        for i in self.pool_list:
            pool=int(math.ceil(shape[1]/i))
            stride=int(math.floor(shape[1]/i))
            x=MaxPooling2D((pool,pool),strides=(stride,stride))(inputs)
            x=self.flatten(x)
            self.output_list.append(x)
        output=self.concat(self.output_list)
        return output

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('./dense_spp.h5',custom_objects={'SPP':SPP})

In [ ]:
model.evaluate(valid)

In [ ]:
test=pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
sub=pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
inf_model=load_model('dense.h5')

In [ ]:
inf_model.evaluate(valid)

In [ ]:
test['diagnosis']=-1
for i in tqdm(range(test.shape[0])):
    imgpath='../input/aptos2019-blindness-detection/test_images/'+str(test.iloc[i,0])+'.png'
    image=next(preprocess(imgpath))
    img=cv2.resize(image,(900,900))
    img=img_to_array(img)/255.0
    img=np.expand_dims(img,axis=0)
    predictions=inf_model.predict(img)
    test.loc[i,'diagnosis']=np.argmax(predictions,axis=1)[0]
    

In [ ]:
test.head()

In [ ]:
test['diagnosis'].value_counts()

In [ ]:
sub.head()

In [ ]:
test.to_csv('submission.csv',index=False)

In [ ]:
#plt.figure(figsize=(12,5))
#plt.plot(model.history.history.get('loss'),label='loss',color='red')
#plt.plot(model.history.history.get('val_loss'),label='val_loss',color='green')
#plt.grid(alpha=0.4)
#plt.legend(loc=0)
#plt.show()

In [ ]:
#plt.figure(figsize=(12,5))
#plt.plot(model.history.history.get('auc'),label='auc',color='red')
#plt.plot(model.history.history.get('val_auc'),label='val_auc',color='green')
#plt.grid(alpha=0.4)
#plt.legend(loc=0)
#plt.show()